![banner_etl](https://github.com/cistelsa/Commerce_Data_Analysis_and_Recommendations/blob/main/5_Sources/Images/banner_automatizacion.gif?raw=true)

## **Normalización tablas Yelp**
#### `tips_yelp`, `categories_yelp`, `categories_hotels_yelp`, `reviews_yelp`, `pre_users_yelp`

**<mark style="background:#2bfe9c">Script de automatización</mark> <mark>Normalización de Tablas</mark>**

In [1]:
# Librerías necesarias para este notebook de automatización
import pandas as pd
import re

StatementMeta(, 966d8644-034b-4f9b-be6e-fb8b23502e55, 3, Finished, Available)

#### **Se aplica el Diagrama de Entidad Relación diseñado en el gestor de proyectos [Aquí](https://github.com/cistelsa/Commerce_Data_Analysis_and_Recommendations/issues/27#issuecomment-1756480203)**

## **Yelp**

In [2]:
# Leer el dataset hotels_yelp_hb
df_hotels_yelp = pd.read_csv("/lakehouse/default/Files/data/beta/Yelp/hotels_yelp_hb.csv")

StatementMeta(, 966d8644-034b-4f9b-be6e-fb8b23502e55, 4, Finished, Available)

In [3]:
df_hotels_yelp.info()

StatementMeta(, 966d8644-034b-4f9b-be6e-fb8b23502e55, 5, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1593 entries, 0 to 1592
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   hotel_id      1593 non-null   int64  
 1   business_id   1593 non-null   object 
 2   name          1593 non-null   object 
 3   address       1593 non-null   object 
 4   city          1593 non-null   object 
 5   postal_code   1593 non-null   int64  
 6   latitude      1593 non-null   float64
 7   longitude     1593 non-null   float64
 8   stars         1593 non-null   float64
 9   review_count  1593 non-null   int64  
 10  is_open       1593 non-null   int64  
 11  attributes    1487 non-null   object 
 12  categories    1593 non-null   object 
 13  hours         1234 non-null   object 
 14  state         1593 non-null   object 
dtypes: float64(3), int64(4), object(8)
memory usage: 186.8+ KB


In [4]:
# Leer el dataset hotels_yelp_hb
df_tips_yelp = pd.read_csv("/lakehouse/default/Files/data/beta/Yelp/tip.csv")

StatementMeta(, 966d8644-034b-4f9b-be6e-fb8b23502e55, 6, Finished, Available)

In [5]:
df_tips_yelp.info()

StatementMeta(, 966d8644-034b-4f9b-be6e-fb8b23502e55, 7, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7748 entries, 0 to 7747
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   business_id       7748 non-null   object
 1   user_id           7748 non-null   object
 2   text              7748 non-null   object
 3   date              7748 non-null   object
 4   compliment_count  7748 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 302.8+ KB


In [6]:
df_tips_copy_yelp = df_tips_yelp.copy()

StatementMeta(, 966d8644-034b-4f9b-be6e-fb8b23502e55, 8, Finished, Available)

In [7]:
df_tips_copy_yelp = df_tips_copy_yelp.merge(df_hotels_yelp[['business_id', 'hotel_id']], on='business_id', how='inner')

StatementMeta(, 966d8644-034b-4f9b-be6e-fb8b23502e55, 9, Finished, Available)

#### **Tabla `tips_yelp`**

In [8]:
# Organizamos las columnas y cambiamos nombres para seguir nuestro ERD
df_tips_copy_yelp = df_tips_copy_yelp[['hotel_id', 'user_id', 'text', 'date']].rename(columns={'text':'tip_yelp_text', 'date':'tip_yelp_date'})

StatementMeta(, 966d8644-034b-4f9b-be6e-fb8b23502e55, 10, Finished, Available)

In [9]:
# Convertir a spark para guardar en formato parquet
df_tips_s = spark.createDataFrame(df_tips_copy_yelp)

StatementMeta(, 966d8644-034b-4f9b-be6e-fb8b23502e55, 11, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


In [10]:
# Guardar el DataFrame en un solo archivo Parquet
# Opcional: Sobrescribir si el archivo ya existe
df_tips_s.coalesce(3).write \
    .format("parquet") \
    .mode("overwrite") \
    .save("Files/data/launch/Yelp/tips_yelp.parquet")

StatementMeta(, 966d8644-034b-4f9b-be6e-fb8b23502e55, 12, Submitted, Running)

#### **Tabla `reviews_yelp`**

In [5]:
df_reviews = spark.read.parquet("Files/data/beta/Yelp/review.parquet")

StatementMeta(, c456b074-d38b-4960-84b5-696a321c04fe, 7, Finished, Available)

In [11]:
df_reviews_p = df_reviews.toPandas()

StatementMeta(, c456b074-d38b-4960-84b5-696a321c04fe, 13, Finished, Available)

In [12]:
df_reviews_p = df_reviews_p.merge(df_hotels_yelp[['business_id', 'hotel_id']], on='business_id', how='inner')

StatementMeta(, c456b074-d38b-4960-84b5-696a321c04fe, 14, Finished, Available)

In [13]:
df_reviews_p.info()

StatementMeta(, c456b074-d38b-4960-84b5-696a321c04fe, 15, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89185 entries, 0 to 89184
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   business_id  89185 non-null  object        
 1   review_id    89185 non-null  object        
 2   user_id      89185 non-null  object        
 3   stars        89185 non-null  float64       
 4   useful       89185 non-null  int32         
 5   funny        89185 non-null  int32         
 6   cool         89185 non-null  int32         
 7   text         89185 non-null  object        
 8   date         89185 non-null  datetime64[ns]
 9   hotel_id     89185 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(3), int64(1), object(4)
memory usage: 6.5+ MB


In [14]:
# Estructura de la tabla reviews_gm
df_reviews_p = df_reviews_p.loc[:, ['business_id','hotel_id', 'user_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']].rename(columns={'business_id':'review_id', 'text':'review_yelp_text', 'date':'review_yelp_date'})

StatementMeta(, c456b074-d38b-4960-84b5-696a321c04fe, 16, Finished, Available)

In [15]:
# Reemplaza caracteres especiales y convierte a minúsculas para el procesamiento de ML.
def clean_and_lowercase(column):
    column = column.str.replace('[^\w\s]', ' ')  # Remove special characters
    column = column.str.lower()  # Convert to lowercase
    return column

df_reviews_p['review_yelp_text'] = clean_and_lowercase(df_reviews_p['review_yelp_text'])

StatementMeta(, c456b074-d38b-4960-84b5-696a321c04fe, 17, Finished, Available)

/tmp/ipykernel_8388/2579608439.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  column = column.str.replace('[^\w\s]', ' ')  # Remove special characters


In [16]:
df_reviews_s = spark.createDataFrame(df_reviews_p)

StatementMeta(, c456b074-d38b-4960-84b5-696a321c04fe, 18, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


In [17]:
# Guardar el DataFrame en un solo archivo Parquet
# Opcional: Sobrescribir si el archivo ya existe
df_reviews_s.coalesce(3).write \
    .format("parquet") \
    .mode("overwrite") \
    .save("Files/data/launch/Yelp/reviews_yelp.parquet")

StatementMeta(, c456b074-d38b-4960-84b5-696a321c04fe, 19, Finished, Available)

In [18]:
df_reviews_p.info()

StatementMeta(, c456b074-d38b-4960-84b5-696a321c04fe, 20, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89185 entries, 0 to 89184
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   review_id         89185 non-null  object        
 1   hotel_id          89185 non-null  int64         
 2   user_id           89185 non-null  object        
 3   stars             89185 non-null  float64       
 4   useful            89185 non-null  int32         
 5   funny             89185 non-null  int32         
 6   cool              89185 non-null  int32         
 7   review_yelp_text  89185 non-null  object        
 8   review_yelp_date  89185 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int32(3), int64(1), object(3)
memory usage: 5.8+ MB


#### **Tabla `categories_yelp`**

In [23]:
# Creamos una copia del dataframe principal de df_hotels_gm
df_hotels_copy_to_category_yelp = df_hotels_yelp.copy()

StatementMeta(, 46dc7621-8d9a-4155-ba08-777c42bf59de, 25, Finished, Available)

In [24]:
df_hotels_copy_to_category_yelp.info()

StatementMeta(, 46dc7621-8d9a-4155-ba08-777c42bf59de, 26, Finished, Available)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1593 entries, 0 to 1592
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   hotel_id      1593 non-null   int64  
 1   business_id   1593 non-null   object 
 2   name          1593 non-null   object 
 3   address       1593 non-null   object 
 4   city          1593 non-null   object 
 5   postal_code   1593 non-null   int64  
 6   latitude      1593 non-null   float64
 7   longitude     1593 non-null   float64
 8   stars         1593 non-null   float64
 9   review_count  1593 non-null   int64  
 10  is_open       1593 non-null   int64  
 11  attributes    1487 non-null   object 
 12  categories    1593 non-null   object 
 13  hours         1234 non-null   object 
 14  state         1593 non-null   object 
dtypes: float64(3), int64(4), object(8)
memory usage: 186.8+ KB


In [26]:
# Convertimos la columna a listas de python
df_hotels_copy_to_category_yelp['categories'] = df_hotels_copy_to_category_yelp['categories'].apply(eval)
# Desanidar la columna 'categories'
df_pre_categories_yelp = df_hotels_copy_to_category_yelp.explode('categories', ignore_index=True)

StatementMeta(, 46dc7621-8d9a-4155-ba08-777c42bf59de, 28, Finished, Available)

In [28]:
# Creamos un dataframe para generar el datset categories_yelp
df_pre_categories_id_yelp = pd.DataFrame()
# Creamos la columna categories que se iguale al dataframe df_pre_categories_yelp
df_pre_categories_id_yelp['categories'] = df_pre_categories_yelp['categories']

StatementMeta(, 46dc7621-8d9a-4155-ba08-777c42bf59de, 30, Finished, Available)

In [29]:
# Eliminamos duplicados
df_pre_categories_id_yelp.drop_duplicates(subset=['categories'], inplace=True)
# Creamos un id para la columna category_id
df_pre_categories_id_yelp['category_id'] = range(1001, 1001 + len(df_pre_categories_id_yelp))

StatementMeta(, 46dc7621-8d9a-4155-ba08-777c42bf59de, 31, Finished, Available)

In [30]:
# Organizamos las columnas y cambiamos nombres para seguir nuestro ERD
df_pre_categories_id_yelp = df_pre_categories_id_yelp[['category_id', 'categories']].rename(columns={'categories':'category_name'})

StatementMeta(, 46dc7621-8d9a-4155-ba08-777c42bf59de, 32, Finished, Available)

In [31]:
# Creamos el dataset ya normalizado categories_yelp.csv
df_pre_categories_id_yelp.to_csv("/lakehouse/default/Files/data/launch/Yelp/categories_yelp.csv", index=False)

StatementMeta(, 46dc7621-8d9a-4155-ba08-777c42bf59de, 33, Finished, Available)

#### **Tabla `categories_hotels_yelp`**

In [32]:
# Realizamos un Inner Join para asociar las categorías por hotel_id
df_pre_categories_hotels_yelp = df_pre_categories_id_yelp.merge(df_pre_categories_yelp[['categories', 'hotel_id']], left_on='category_name', right_on='categories', how='inner')

StatementMeta(, 46dc7621-8d9a-4155-ba08-777c42bf59de, 34, Finished, Available)

In [33]:
# Organizamos las columnas como en el ERD
df_pre_categories_hotels_yelp = df_pre_categories_hotels_yelp[['hotel_id', 'category_id']]

StatementMeta(, 46dc7621-8d9a-4155-ba08-777c42bf59de, 35, Finished, Available)

In [34]:
# Creamos el dataset normalizado categories_hotels_yelp.csv
df_pre_categories_hotels_yelp.to_csv("/lakehouse/default/Files/data/launch/Yelp/categories_hotels_yelp.csv", index=False)

StatementMeta(, 46dc7621-8d9a-4155-ba08-777c42bf59de, 36, Finished, Available)

#### **Tabla `checkin`**

In [19]:
df_checkin = pd.read_csv("/lakehouse/default/Files/data/beta/Yelp/checkin.csv")

StatementMeta(, c456b074-d38b-4960-84b5-696a321c04fe, 21, Finished, Available)

In [21]:
# Divide la cadena en una lista y desanida las fechas y horas.
df_checkin['date'] = df_checkin['date'].str.split(', ')
df_checkin = df_checkin.explode('date', ignore_index=True)

StatementMeta(, c456b074-d38b-4960-84b5-696a321c04fe, 23, Finished, Available)

In [23]:
# Realizamos un Inner Join para asociar el checkin por hotel_id
df_checkin = df_checkin.merge(df_hotels_yelp[['business_id', 'hotel_id']], on='business_id', how='inner')

StatementMeta(, c456b074-d38b-4960-84b5-696a321c04fe, 25, Finished, Available)

In [25]:
# Organizamos las columnas como en el ERD
df_checkin = df_checkin[['hotel_id', 'date']].rename(columns={'date':'checkin_date'})

StatementMeta(, c456b074-d38b-4960-84b5-696a321c04fe, 27, Finished, Available)

In [27]:
# Creamos el dataset normalizado checkin.csv
df_checkin.to_csv("/lakehouse/default/Files/data/launch/Yelp/checkin.csv", index=False)

StatementMeta(, c456b074-d38b-4960-84b5-696a321c04fe, 29, Finished, Available)